# This code cleans the Divvy Trip data 

It removes duplicates, trips under 5 minutes, trips over 1 day, and columns that will not be used in the capstone. 
It then exports the remaining data to a csv called "bikes_clean"  

In [1]:
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
import re

In [2]:
bikes = pd.read_csv('data/Chicago/Divvy_trips.csv') #reads divvy trip information as bikes

In [3]:
bikes

,TRIP ID,START TIME,STOP TIME,BIKE ID,TRIP DURATION,FROM STATION ID,FROM STATION NAME,TO STATION ID,TO STATION NAME,USER TYPE,GENDER,BIRTH YEAR,FROM LATITUDE,FROM LONGITUDE,FROM LOCATION,TO LATITUDE,TO LONGITUDE,TO LOCATION
0,8546790,12/31/2015 05:35:00 PM,12/31/2015 05:44:00 PM,979,521,117,Wilton Ave & Belmont Ave,229,Southport Ave & Roscoe St,Subscriber,Female,1991.0,41.940180,-87.653040,POINT (-87.65304 41.94018),41.943739,-87.664020,POINT (-87.66402 41.943739)
1,8546793,12/31/2015 05:37:00 PM,12/31/2015 05:41:00 PM,1932,256,301,Clark St & Schiller St,138,Clybourn Ave & Division St,Subscriber,Male,1992.0,41.907993,-87.631501,POINT (-87.631501 41.907993),41.904613,-87.640552,POINT (-87.640552 41.904613)
2,8546795,12/31/2015 05:37:00 PM,12/31/2015 05:40:00 PM,1693,134,465,Marine Dr & Ainslie St,251,Clarendon Ave & Leland Ave,Subscriber,Female,1987.0,41.971600,-87.650154,POINT (-87.650154 41.9716),41.967968,-87.650001,POINT (-87.650001 41.967968)
3,8546797,12/31/2015 05:38:00 PM,12/31/2015 05:55:00 PM,3370,995,333,Ashland Ave & Blackhawk St,198,Green St (Halsted St) & Madison St,Subscriber,Male,1975.0,41.907066,-87.667252,POINT (-87.667252 41.907066),41.881892,-87.648789,POINT (-87.648789 41.881892)
4,8546798,12/31/2015 05:38:00 PM,12/31/2015 05:41:00 PM,2563,177,48,Larrabee St & Kingsbury St,111,Sedgwick St & Huron St,Subscriber,Male,1990.0,41.897764,-87.642884,POINT (-87.642884 41.897764),41.894666,-87.638437,POINT (-87.638437 41.894666)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21242735,25220103,09/30/2019 06:24:15 PM,09/30/2019 06:35:26 PM,6352,671,60,Dayton St & North Ave,332,Burling St (Halsted) & Diversey Pkwy (Temp),Subscriber,Female,1970.0,41.910578,-87.649422,POINT (-87.6494219288 41.9105780349),41.933140,-87.647760,POINT (-87.64776 41.93314)
21242736,25221957,09/30/2019 07:46:32 PM,09/30/2019 07:52:07 PM,2485,335,110,Dearborn St & Erie St,38,Clark St & Lake St,Subscriber,Male,1991.0,41.893992,-87.629318,POINT (-87.629318 41.893992),41.886021,-87.630876,POINT (-87.6308760584 41.88602082773)
21242737,25223072,09/30/2019 09:35:55 PM,09/30/2019 09:56:41 PM,4305,1246,69,Damen Ave & Pierce Ave,347,Ashland Ave & Grace St,Subscriber,Male,1974.0,41.909396,-87.677692,POINT (-87.6776919292 41.9093960065),41.950687,-87.668700,POINT (-87.6687 41.950687)
21242738,25219153,09/30/2019 05:58:35 PM,09/30/2019 06:10:05 PM,2373,690,211,St. Clair St & Erie St,289,Wells St & Concord Ln,Subscriber,Female,1992.0,41.894448,-87.622663,POINT (-87.622663 41.894448),41.912133,-87.634656,POINT (-87.634656 41.912133)


In [4]:
bikes.shape

(21242740, 18)

In [5]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21242740 entries, 0 to 21242739
Data columns (total 18 columns):
 #   Column             Dtype  
---  ------             -----  
 0   TRIP ID            int64  
 1   START TIME         object 
 2   STOP TIME          object 
 3   BIKE ID            int64  
 4   TRIP DURATION      int64  
 5   FROM STATION ID    int64  
 6   FROM STATION NAME  object 
 7   TO STATION ID      int64  
 8   TO STATION NAME    object 
 9   USER TYPE          object 
 10  GENDER             object 
 11  BIRTH YEAR         float64
 12  FROM LATITUDE      float64
 13  FROM LONGITUDE     float64
 14  FROM LOCATION      object 
 15  TO LATITUDE        float64
 16  TO LONGITUDE       float64
 17  TO LOCATION        object 
dtypes: float64(5), int64(5), object(8)
memory usage: 2.8+ GB


In [6]:
bikes.describe()

,TRIP ID,BIKE ID,TRIP DURATION,FROM STATION ID,TO STATION ID,BIRTH YEAR,FROM LATITUDE,FROM LONGITUDE,TO LATITUDE,TO LONGITUDE
count,2.124274e+07,2.124274e+07,2.124274e+07,2.124274e+07,2.124274e+07,1.637603e+07,2.124248e+07,2.124248e+07,2.124206e+07,2.124206e+07
mean,1.353282e+07,2.900286e+03,1.151073e+03,1.800243e+02,1.804185e+02,1.981430e+03,4.189910e+01,-8.764185e+01,4.189963e+01,-8.764204e+01
std,7.458796e+06,1.813873e+03,1.819884e+04,1.320127e+02,1.321821e+02,1.093585e+01,3.520141e-02,2.231165e-02,3.539643e-02,2.243568e-02
min,3.940000e+03,1.000000e+00,6.000000e+01,1.000000e+00,1.000000e+00,1.759000e+03,4.173665e+01,-8.780287e+01,4.173665e+01,-8.780287e+01
25%,7.186379e+06,1.346000e+03,4.110000e+02,7.500000e+01,7.500000e+01,1.976000e+03,4.188096e+01,-8.765357e+01,4.188096e+01,-8.765371e+01
50%,1.375492e+07,2.705000e+03,7.070000e+02,1.570000e+02,1.570000e+02,1.984000e+03,4.189186e+01,-8.763874e+01,4.189228e+01,-8.763889e+01
75%,2.000726e+07,4.387000e+03,1.210000e+03,2.680000e+02,2.680000e+02,1.989000e+03,4.191831e+01,-8.762680e+01,4.191844e+01,-8.762680e+01
max,2.596290e+07,6.946000e+03,1.345320e+07,6.730000e+02,6.730000e+02,2.017000e+03,4.206485e+01,-8.754939e+01,4.206485e+01,-8.754939e+01


In [7]:
test = bikes.duplicated(subset=['BIKE ID','START TIME','STOP TIME'], keep='first') 
test #tests to see if there are duplicates (where bike id, start time, and stop time are the same)

0           False
1           False
2           False
3           False
4           False
            ...  
21242735    False
21242736    False
21242737    False
21242738    False
21242739    False
Length: 21242740, dtype: bool

In [8]:
test.value_counts() #shows that there are 189 duplicate values

False    21242551
True          189
dtype: int64

In [9]:
bikes.loc[(bikes.duplicated(subset=['BIKE ID','START TIME','STOP TIME'], keep='first') == True)] #displays the duplicate values

,TRIP ID,START TIME,STOP TIME,BIKE ID,TRIP DURATION,FROM STATION ID,FROM STATION NAME,TO STATION ID,TO STATION NAME,USER TYPE,GENDER,BIRTH YEAR,FROM LATITUDE,FROM LONGITUDE,FROM LOCATION,TO LATITUDE,TO LONGITUDE,TO LOCATION
968670,223381,08/14/2013 11:41:00 AM,08/14/2013 11:45:00 AM,1184,246,100,Orleans St & Merchandise Mart Plaza,47,State St & Kinzie St,Subscriber,Male,1966.0,41.888243,-87.636390,POINT (-87.63639 41.888243),41.889180,-87.627700,POINT (-87.6277 41.88918)
2553768,4117530,10/30/2014 10:24:00 AM,10/30/2014 10:36:00 AM,2083,680,176,Clark St & Elm St,66,Clinton St & Lake St,Subscriber,Male,1981.0,41.903233,-87.631253,POINT (-87.631253 41.903233),41.885464,-87.641253,POINT (-87.641253 41.885464)
3117622,4890792,04/17/2015 11:30:00 AM,04/17/2015 11:33:00 AM,2544,175,51,Clark St & Randolph St,49,Dearborn St & Monroe St,Subscriber,Male,1972.0,41.884576,-87.631890,POINT (-87.63189 41.884576),41.881320,-87.629521,POINT (-87.629521 41.88132)
3999669,5938747,06/30/2015 06:21:00 PM,06/30/2015 06:47:00 PM,3756,1545,35,Streeter Dr & Illinois St,214,Damen Ave & Grand Ave,Subscriber,Female,1987.0,41.891071,-87.612200,POINT (-87.6122 41.891071),41.891220,-87.676860,POINT (-87.67686 41.89122)
4237572,621155,09/28/2013 04:32:00 PM,09/28/2013 05:39:00 PM,181,4028,313,Lakeview Ave & Fullerton Pkwy,313,Lakeview Ave & Fullerton Pkwy,Customer,NaN,NaN,41.925858,-87.638973,POINT (-87.638973 41.925858),41.925858,-87.638973,POINT (-87.638973 41.925858)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16378479,23688546,07/11/2019 04:49:59 PM,07/11/2019 05:15:24 PM,4364,1524,198,Green St & Madison St,268,Lake Shore Dr & North Blvd,Subscriber,Female,1994.0,41.881892,-87.648789,POINT (-87.648789 41.881892),41.911722,-87.626804,POINT (-87.626804 41.911722)
16378545,23698561,07/11/2019 07:46:08 PM,07/11/2019 08:04:20 PM,3748,1092,220,Clark St & Drummond Pl,176,Clark St & Elm St,Subscriber,Male,1988.0,41.931248,-87.644336,POINT (-87.644336 41.931248),41.902973,-87.631280,POINT (-87.63128 41.902973)
16378625,23692529,07/11/2019 05:42:52 PM,07/11/2019 05:50:36 PM,398,463,108,Halsted St & Polk St,135,Halsted St & 21st St,Subscriber,Female,1996.0,41.871840,-87.646640,POINT (-87.64664 41.87184),41.853780,-87.646650,POINT (-87.64665 41.85378)
16378647,23693585,07/11/2019 05:37:30 PM,07/11/2019 05:51:11 PM,1485,821,220,Clark St & Drummond Pl,620,Orleans St & Chestnut St (NEXT Apts),Subscriber,Male,1993.0,41.931248,-87.644336,POINT (-87.644336 41.931248),41.898203,-87.637536,POINT (-87.637536 41.898203)


In [10]:
bikes_clean = bikes.loc[(bikes.duplicated(subset=['BIKE ID','START TIME','STOP TIME'], keep='first') == False)] 
#removes duplicate values saves under bikes_clean

In [11]:
#creates smaller dataset to test functions
test10000 = bikes_clean.head(10000)

In [12]:
test10000['start_dt'] = pd.to_datetime(test10000['START TIME'].astype(str))
test10000['end_dt'] = pd.to_datetime(test10000['STOP TIME'].astype(str))

test10000

C:\Users\Diarra\AppData\Local\Temp\ipykernel_20360\3853639680.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test10000['start_dt'] = pd.to_datetime(test10000['START TIME'].astype(str))
C:\Users\Diarra\AppData\Local\Temp\ipykernel_20360\3853639680.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test10000['end_dt'] = pd.to_datetime(test10000['STOP TIME'].astype(str))


,TRIP ID,START TIME,STOP TIME,BIKE ID,TRIP DURATION,FROM STATION ID,FROM STATION NAME,TO STATION ID,TO STATION NAME,USER TYPE,GENDER,BIRTH YEAR,FROM LATITUDE,FROM LONGITUDE,FROM LOCATION,TO LATITUDE,TO LONGITUDE,TO LOCATION,start_dt,end_dt
0,8546790,12/31/2015 05:35:00 PM,12/31/2015 05:44:00 PM,979,521,117,Wilton Ave & Belmont Ave,229,Southport Ave & Roscoe St,Subscriber,Female,1991.0,41.940180,-87.653040,POINT (-87.65304 41.94018),41.943739,-87.664020,POINT (-87.66402 41.943739),2015-12-31 17:35:00,2015-12-31 17:44:00
1,8546793,12/31/2015 05:37:00 PM,12/31/2015 05:41:00 PM,1932,256,301,Clark St & Schiller St,138,Clybourn Ave & Division St,Subscriber,Male,1992.0,41.907993,-87.631501,POINT (-87.631501 41.907993),41.904613,-87.640552,POINT (-87.640552 41.904613),2015-12-31 17:37:00,2015-12-31 17:41:00
2,8546795,12/31/2015 05:37:00 PM,12/31/2015 05:40:00 PM,1693,134,465,Marine Dr & Ainslie St,251,Clarendon Ave & Leland Ave,Subscriber,Female,1987.0,41.971600,-87.650154,POINT (-87.650154 41.9716),41.967968,-87.650001,POINT (-87.650001 41.967968),2015-12-31 17:37:00,2015-12-31 17:40:00
3,8546797,12/31/2015 05:38:00 PM,12/31/2015 05:55:00 PM,3370,995,333,Ashland Ave & Blackhawk St,198,Green St (Halsted St) & Madison St,Subscriber,Male,1975.0,41.907066,-87.667252,POINT (-87.667252 41.907066),41.881892,-87.648789,POINT (-87.648789 41.881892),2015-12-31 17:38:00,2015-12-31 17:55:00
4,8546798,12/31/2015 05:38:00 PM,12/31/2015 05:41:00 PM,2563,177,48,Larrabee St & Kingsbury St,111,Sedgwick St & Huron St,Subscriber,Male,1990.0,41.897764,-87.642884,POINT (-87.642884 41.897764),41.894666,-87.638437,POINT (-87.638437 41.894666),2015-12-31 17:38:00,2015-12-31 17:41:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1014165,11/22/2013 04:15:00 PM,11/22/2013 04:19:00 PM,1313,290,49,Dearborn St & Monroe St,91,Clinton St & Washington Blvd,Subscriber,Male,1979.0,41.881320,-87.629521,POINT (-87.629521 41.88132),41.883380,-87.641170,POINT (-87.64117 41.88338),2013-11-22 16:15:00,2013-11-22 16:19:00
9996,1014168,11/22/2013 04:15:00 PM,11/22/2013 04:32:00 PM,1177,1020,286,Franklin St & Quincy St,113,Bissell St & Armitage Ave,Subscriber,Male,1976.0,41.878724,-87.634793,POINT (-87.634793 41.878724),41.918440,-87.652220,POINT (-87.65222 41.91844),2013-11-22 16:15:00,2013-11-22 16:32:00
9997,1014169,11/22/2013 04:16:00 PM,11/22/2013 04:25:00 PM,1554,576,32,Racine Ave & Congress Pkwy,84,Green St & Milwaukee Ave,Subscriber,Male,1983.0,41.874640,-87.657030,POINT (-87.65703 41.87464),41.891000,-87.645925,POINT (-87.645925 41.891),2013-11-22 16:16:00,2013-11-22 16:25:00
9998,1014171,11/22/2013 04:13:00 PM,11/22/2013 04:22:00 PM,2603,533,52,Michigan Ave & Lake St,50,Clark St & Congress Pkwy,Subscriber,Male,1977.0,41.886024,-87.624117,POINT (-87.624117 41.886024),41.875933,-87.630585,POINT (-87.630585 41.875933),2013-11-22 16:13:00,2013-11-22 16:22:00


In [13]:
test10000['trip_length'] = (test10000.end_dt-test10000.start_dt).dt.total_seconds() #calulates duration from timestamp to ensure it matches the documented trip duration 
test10000

C:\Users\Diarra\AppData\Local\Temp\ipykernel_20360\437175840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test10000['trip_length'] = (test10000.end_dt-test10000.start_dt).dt.total_seconds() #calulates duration from timestamp to ensure it matches the documented trip duration


,TRIP ID,START TIME,STOP TIME,BIKE ID,TRIP DURATION,FROM STATION ID,FROM STATION NAME,TO STATION ID,TO STATION NAME,USER TYPE,...,BIRTH YEAR,FROM LATITUDE,FROM LONGITUDE,FROM LOCATION,TO LATITUDE,TO LONGITUDE,TO LOCATION,start_dt,end_dt,trip_length
0,8546790,12/31/2015 05:35:00 PM,12/31/2015 05:44:00 PM,979,521,117,Wilton Ave & Belmont Ave,229,Southport Ave & Roscoe St,Subscriber,...,1991.0,41.940180,-87.653040,POINT (-87.65304 41.94018),41.943739,-87.664020,POINT (-87.66402 41.943739),2015-12-31 17:35:00,2015-12-31 17:44:00,540.0
1,8546793,12/31/2015 05:37:00 PM,12/31/2015 05:41:00 PM,1932,256,301,Clark St & Schiller St,138,Clybourn Ave & Division St,Subscriber,...,1992.0,41.907993,-87.631501,POINT (-87.631501 41.907993),41.904613,-87.640552,POINT (-87.640552 41.904613),2015-12-31 17:37:00,2015-12-31 17:41:00,240.0
2,8546795,12/31/2015 05:37:00 PM,12/31/2015 05:40:00 PM,1693,134,465,Marine Dr & Ainslie St,251,Clarendon Ave & Leland Ave,Subscriber,...,1987.0,41.971600,-87.650154,POINT (-87.650154 41.9716),41.967968,-87.650001,POINT (-87.650001 41.967968),2015-12-31 17:37:00,2015-12-31 17:40:00,180.0
3,8546797,12/31/2015 05:38:00 PM,12/31/2015 05:55:00 PM,3370,995,333,Ashland Ave & Blackhawk St,198,Green St (Halsted St) & Madison St,Subscriber,...,1975.0,41.907066,-87.667252,POINT (-87.667252 41.907066),41.881892,-87.648789,POINT (-87.648789 41.881892),2015-12-31 17:38:00,2015-12-31 17:55:00,1020.0
4,8546798,12/31/2015 05:38:00 PM,12/31/2015 05:41:00 PM,2563,177,48,Larrabee St & Kingsbury St,111,Sedgwick St & Huron St,Subscriber,...,1990.0,41.897764,-87.642884,POINT (-87.642884 41.897764),41.894666,-87.638437,POINT (-87.638437 41.894666),2015-12-31 17:38:00,2015-12-31 17:41:00,180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1014165,11/22/2013 04:15:00 PM,11/22/2013 04:19:00 PM,1313,290,49,Dearborn St & Monroe St,91,Clinton St & Washington Blvd,Subscriber,...,1979.0,41.881320,-87.629521,POINT (-87.629521 41.88132),41.883380,-87.641170,POINT (-87.64117 41.88338),2013-11-22 16:15:00,2013-11-22 16:19:00,240.0
9996,1014168,11/22/2013 04:15:00 PM,11/22/2013 04:32:00 PM,1177,1020,286,Franklin St & Quincy St,113,Bissell St & Armitage Ave,Subscriber,...,1976.0,41.878724,-87.634793,POINT (-87.634793 41.878724),41.918440,-87.652220,POINT (-87.65222 41.91844),2013-11-22 16:15:00,2013-11-22 16:32:00,1020.0
9997,1014169,11/22/2013 04:16:00 PM,11/22/2013 04:25:00 PM,1554,576,32,Racine Ave & Congress Pkwy,84,Green St & Milwaukee Ave,Subscriber,...,1983.0,41.874640,-87.657030,POINT (-87.65703 41.87464),41.891000,-87.645925,POINT (-87.645925 41.891),2013-11-22 16:16:00,2013-11-22 16:25:00,540.0
9998,1014171,11/22/2013 04:13:00 PM,11/22/2013 04:22:00 PM,2603,533,52,Michigan Ave & Lake St,50,Clark St & Congress Pkwy,Subscriber,...,1977.0,41.886024,-87.624117,POINT (-87.624117 41.886024),41.875933,-87.630585,POINT (-87.630585 41.875933),2013-11-22 16:13:00,2013-11-22 16:22:00,540.0


In [14]:
test10000['difference'] = test10000.trip_length-test10000['TRIP DURATION'] #calulated duration seems to be different from duration documented
test10000

C:\Users\Diarra\AppData\Local\Temp\ipykernel_20360\4200335961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test10000['difference'] = test10000.trip_length-test10000['TRIP DURATION'] #calulated duration seems to be different from duration documented


,TRIP ID,START TIME,STOP TIME,BIKE ID,TRIP DURATION,FROM STATION ID,FROM STATION NAME,TO STATION ID,TO STATION NAME,USER TYPE,...,FROM LATITUDE,FROM LONGITUDE,FROM LOCATION,TO LATITUDE,TO LONGITUDE,TO LOCATION,start_dt,end_dt,trip_length,difference
0,8546790,12/31/2015 05:35:00 PM,12/31/2015 05:44:00 PM,979,521,117,Wilton Ave & Belmont Ave,229,Southport Ave & Roscoe St,Subscriber,...,41.940180,-87.653040,POINT (-87.65304 41.94018),41.943739,-87.664020,POINT (-87.66402 41.943739),2015-12-31 17:35:00,2015-12-31 17:44:00,540.0,1.900000e+01
1,8546793,12/31/2015 05:37:00 PM,12/31/2015 05:41:00 PM,1932,256,301,Clark St & Schiller St,138,Clybourn Ave & Division St,Subscriber,...,41.907993,-87.631501,POINT (-87.631501 41.907993),41.904613,-87.640552,POINT (-87.640552 41.904613),2015-12-31 17:37:00,2015-12-31 17:41:00,240.0,-1.600000e+01
2,8546795,12/31/2015 05:37:00 PM,12/31/2015 05:40:00 PM,1693,134,465,Marine Dr & Ainslie St,251,Clarendon Ave & Leland Ave,Subscriber,...,41.971600,-87.650154,POINT (-87.650154 41.9716),41.967968,-87.650001,POINT (-87.650001 41.967968),2015-12-31 17:37:00,2015-12-31 17:40:00,180.0,4.600000e+01
3,8546797,12/31/2015 05:38:00 PM,12/31/2015 05:55:00 PM,3370,995,333,Ashland Ave & Blackhawk St,198,Green St (Halsted St) & Madison St,Subscriber,...,41.907066,-87.667252,POINT (-87.667252 41.907066),41.881892,-87.648789,POINT (-87.648789 41.881892),2015-12-31 17:38:00,2015-12-31 17:55:00,1020.0,2.500000e+01
4,8546798,12/31/2015 05:38:00 PM,12/31/2015 05:41:00 PM,2563,177,48,Larrabee St & Kingsbury St,111,Sedgwick St & Huron St,Subscriber,...,41.897764,-87.642884,POINT (-87.642884 41.897764),41.894666,-87.638437,POINT (-87.638437 41.894666),2015-12-31 17:38:00,2015-12-31 17:41:00,180.0,3.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1014165,11/22/2013 04:15:00 PM,11/22/2013 04:19:00 PM,1313,290,49,Dearborn St & Monroe St,91,Clinton St & Washington Blvd,Subscriber,...,41.881320,-87.629521,POINT (-87.629521 41.88132),41.883380,-87.641170,POINT (-87.64117 41.88338),2013-11-22 16:15:00,2013-11-22 16:19:00,240.0,-5.000000e+01
9996,1014168,11/22/2013 04:15:00 PM,11/22/2013 04:32:00 PM,1177,1020,286,Franklin St & Quincy St,113,Bissell St & Armitage Ave,Subscriber,...,41.878724,-87.634793,POINT (-87.634793 41.878724),41.918440,-87.652220,POINT (-87.65222 41.91844),2013-11-22 16:15:00,2013-11-22 16:32:00,1020.0,1.136868e-13
9997,1014169,11/22/2013 04:16:00 PM,11/22/2013 04:25:00 PM,1554,576,32,Racine Ave & Congress Pkwy,84,Green St & Milwaukee Ave,Subscriber,...,41.874640,-87.657030,POINT (-87.65703 41.87464),41.891000,-87.645925,POINT (-87.645925 41.891),2013-11-22 16:16:00,2013-11-22 16:25:00,540.0,-3.600000e+01
9998,1014171,11/22/2013 04:13:00 PM,11/22/2013 04:22:00 PM,2603,533,52,Michigan Ave & Lake St,50,Clark St & Congress Pkwy,Subscriber,...,41.886024,-87.624117,POINT (-87.624117 41.886024),41.875933,-87.630585,POINT (-87.630585 41.875933),2013-11-22 16:13:00,2013-11-22 16:22:00,540.0,7.000000e+00


In [15]:
test10000.describe() 
#less that 1 minute difference between seems to indicate that rounding explains the descrepancy in the durations
#will use provided duration

,TRIP ID,BIKE ID,TRIP DURATION,FROM STATION ID,TO STATION ID,BIRTH YEAR,FROM LATITUDE,FROM LONGITUDE,TO LATITUDE,TO LONGITUDE,trip_length,difference
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,8740.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,9.456136e+05,1480.230900,747.159000,144.636500,142.926600,1976.002746,41.894171,-87.641684,41.894284,-87.641361,747.348000,0.189000
std,3.317183e+05,848.278179,1677.563883,95.177143,95.267053,10.814490,0.026869,0.017258,0.026906,0.017326,1678.076828,24.461441
min,1.000000e+04,3.000000,64.000000,5.000000,5.000000,1906.000000,41.788746,-87.707857,41.788746,-87.707857,60.000000,-59.000000
25%,1.002496e+06,721.000000,335.000000,66.000000,59.000000,1969.000000,41.879356,-87.651073,41.880317,-87.651062,360.000000,-17.000000
50%,1.006486e+06,1519.000000,524.000000,114.000000,111.000000,1979.000000,41.886024,-87.639833,41.886024,-87.639833,540.000000,0.000000
75%,1.010316e+06,2202.000000,833.250000,212.000000,212.000000,1985.000000,41.907655,-87.629867,41.907576,-87.629521,840.000000,17.000000
max,8.546813e+06,3370.000000,61509.000000,465.000000,350.000000,1997.000000,41.978353,-87.580715,41.978353,-87.580715,61560.000000,59.000000


In [16]:
bikes_clean.shape

(21242551, 18)

In [17]:
bikes_clean = bikes_clean.loc[(bikes_clean['TRIP DURATION'] >= 300) & (bikes_clean['TRIP DURATION'] <= 86400)] #filters out trips shorter than 5 minutes and longer than 1 day
bikes_clean.shape

(18300905, 18)

In [18]:
bikes_clean = bikes_clean.drop(columns = ['START TIME', 'STOP TIME', 'BIKE ID', 'TRIP DURATION', 'FROM STATION ID', 'FROM STATION NAME', 'TO STATION ID', 'TO STATION NAME', 'USER TYPE', 'GENDER', 'BIRTH YEAR'])
#drops unused columns

In [19]:
bikes_clean

,TRIP ID,FROM LATITUDE,FROM LONGITUDE,FROM LOCATION,TO LATITUDE,TO LONGITUDE,TO LOCATION
0,8546790,41.940180,-87.653040,POINT (-87.65304 41.94018),41.943739,-87.664020,POINT (-87.66402 41.943739)
3,8546797,41.907066,-87.667252,POINT (-87.667252 41.907066),41.881892,-87.648789,POINT (-87.648789 41.881892)
5,8546799,41.837800,-87.651140,POINT (-87.65114 41.8378),41.864883,-87.647071,POINT (-87.647071 41.864883)
7,8546805,41.891072,-87.666611,POINT (-87.666611 41.891072),41.899643,-87.667700,POINT (-87.6677 41.899643)
8,8546813,41.896362,-87.654061,POINT (-87.654061 41.896362),41.890749,-87.632060,POINT (-87.63206 41.890749)
...,...,...,...,...,...,...,...
21242735,25220103,41.910578,-87.649422,POINT (-87.6494219288 41.9105780349),41.933140,-87.647760,POINT (-87.64776 41.93314)
21242736,25221957,41.893992,-87.629318,POINT (-87.629318 41.893992),41.886021,-87.630876,POINT (-87.6308760584 41.88602082773)
21242737,25223072,41.909396,-87.677692,POINT (-87.6776919292 41.9093960065),41.950687,-87.668700,POINT (-87.6687 41.950687)
21242738,25219153,41.894448,-87.622663,POINT (-87.622663 41.894448),41.912133,-87.634656,POINT (-87.634656 41.912133)


In [21]:
bikes_clean.to_csv('data/Chicago/cleaned/bikes_clean.csv')
#exports clean dataset to csv